In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import keras
import tensorflow

In [ ]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/Avenue/training_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('trainer.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
print('Number of frames created :', int(len(imagestore)))

Found  17  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential



#load_model starts here :----------------------------------------------------
def load_model():

	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model

#load_model ends here :----------------------------------------------------



X_train=np.load('trainer.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10 to ease the load_model


frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()


epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')





Trainer has been loaded
Epoch 1/200
22/22 [==============================] - ETA: 0s - loss: 0.2317 - accuracy: 0.5278

22/22 [==============================] - 17s 216ms/step - loss: 0.2317 - accuracy: 0.5278
Epoch 2/200
22/22 [==============================] - ETA: 0s - loss: 0.2066 - accuracy: 0.5448

22/22 [==============================] - 5s 214ms/step - loss: 0.2066 - accuracy: 0.5448
Epoch 3/200
22/22 [==============================] - ETA: 0s - loss: 0.2014 - accuracy: 0.5447

22/22 [==============================] - 6s 273ms/step - loss: 0.2014 - accuracy: 0.5447
Epoch 4/200
22/22 [==============================] - ETA: 0s - loss: 0.1587 - accuracy: 0.6059

22/22 [==============================] - 5s 216ms/step - loss: 0.1587 - accuracy: 0.6059
Epoch 5/200
22/22 [==============================] - ETA: 0s - loss: 0.1178 - accuracy: 0.6786

...

22/22 [==============================] - 5s 233ms/step - loss: 0.0442 - accuracy: 0.7563
Epoch 197/200
22/22 [==============================] - ETA: 0s - loss: 0.0435 - accuracy: 0.7579

22/22 [==============================] - 5s 216ms/step - loss: 0.0435 - accuracy: 0.7579
Epoch 198/200
22/22 [==============================] - ETA: 0s - loss: 0.0428 - accuracy: 0.7588

22/22 [==============================] - 6s 276ms/step - loss: 0.0428 - accuracy: 0.7588
Epoch 199/200
22/22 [==============================] - ETA: 0s - loss: 0.0420 - accuracy: 0.7601

22/22 [==============================] - 5s 213ms/step - loss: 0.0420 - accuracy: 0.7601
Epoch 200/200
22/22 [==============================] - ETA: 0s - loss: 0.0414 - accuracy: 0.7606

22/22 [==============================] - 5s 234ms/step - loss: 0.0414 - accuracy: 0.7606
Done
 Please wait while AnomalyDetector.h5 has been created 
Refresh when needed


In [ ]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/Avenue/testing_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)
#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'
total=0
video_count=0

for video in videos:
		video_count+=1
		print("Video number: ",video_count)
		print("Video:",str(video))
		image_count=0
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		image_count=len(images)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)
		total=len(images)+total
		print("Number of images:",image_count,"\n----------\n")


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('tester.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))

print("Program ended. All testing videos shall be stored in tester.npy \n Please wait while tester.npy is created. \nRefresh when needed")
print('Number of frames created :', int(total))
print ('Number of bunches=',int(total),"/10 = ",int(total/10))
print("\nCorrupted and unreadable bunches were ignored")

Found  21  testing videos
Video number:  1
Video: 01.avi
Number of images: 13 
----------

Video number:  2
Video: 02.avi
Number of images: 13 
----------

Video number:  3
Video: 03.avi
Number of images: 13 
----------

Video number:  4
Video: 04.avi
Number of images: 13 
----------

Video number:  5
Video: 05.avi
Number of images: 13 
----------

Video number:  6
Video: 06.avi
Number of images: 13 
----------

Video number:  7
Video: 07.avi
Number of images: 13 
----------

Video number:  8
Video: 08.avi
Number of images: 13 
----------

Video number:  9
Video: 09.avi
Number of images: 13 
----------

Video number:  10
Video: 10.avi
Number of images: 13 
----------

Video number:  11
Video: 11.avi
Number of images: 13 
----------

Video number:  12
Video: 12.avi
Number of images: 13 
----------

Video number:  13
Video: 13.avi
Number of images: 13 
----------

Video number:  14
Video: 14.avi
Number of images: 13 
----------

Video number:  15
Video: 15.avi
Number of images: 13 
-----

In [ ]:

from keras.models import load_model
import numpy as np 




def mean_squared_loss(x1,x2):




	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist






'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
#threshold=0.0008#(Accuracy level Vishakha)

model=load_model('AnomalyDetector.h5')

X_test=np.load('tester.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
	
	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)



if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")
	
print("\nCorrupted and unreadable bunches were ignored")

1/1 [==============================] - 1s 1s/step
Anomalous bunch of frames at bunch number 0
bunch number:  1
1/1 [==============================] - 0s 68ms/step
Anomalous bunch of frames at bunch number 1
bunch number:  2
1/1 [==============================] - 0s 72ms/step
No anomaly
bunch number:  3
1/1 [==============================] - 0s 64ms/step
No anomaly
bunch number:  4
1/1 [==============================] - 0s 63ms/step
No anomaly
bunch number:  5
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  6
1/1 [==============================] - 0s 66ms/step
No anomaly
bunch number:  7
1/1 [==============================] - 0s 63ms/step
No anomaly
bunch number:  8
1/1 [==============================] - 0s 61ms/step
Anomalous bunch of frames at bunch number 8
bunch number:  9
1/1 [==============================] - 0s 64ms/step
Anomalous bunch of frames at bunch number 9
bunch number:  10
1/1 [==============================] - 0s 64ms/step
No anomaly
bunch 

In [13]:

from keras.models import load_model
import numpy as np 

from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/drive/MyDrive/Avenue/testing_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," files")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'
flag=0
for video in videos:
		if (video=="01.avi"):
			print("Test video found")
			flag=1
			os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
			images=os.listdir(framepath)
			for image in images:
				image_path=framepath+ '/'+ image
				store(image_path)

if flag==0:
	print("Couldn't find test.mp4 or test.avi. Make sure you reupload and try this")
	exit()
imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('sample.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Please wait while video is processed. \nRefresh when needed")


def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between 
	input frame and the reconstructed frame'''


	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist


'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
#threshold=0.0008#(Accuracy level 3)

model=load_model('AnomalyDetector.h5')

X_test=np.load('sample.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
	
	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)


print("----------------------------------------------------\nOUTPUT\n----------------------------------------------------\n")
if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")
	
print("\n----------------------------------------------------\nCorrupted and unreadable bunches were ignored")



Found  22  files
Test video found
Please wait while video is processed. 
Refresh when needed
1/1 [==============================] - 1s 683ms/step
Anomalous bunch of frames at bunch number 0
bunch number:  1
----------------------------------------------------
OUTPUT
----------------------------------------------------

Anomalous Events detected

----------------------------------------------------
Corrupted and unreadable bunches were ignored


In [2]:
import urllib.request

url = "http://www.svcl.ucsd.edu/projects/anomaly/UCSD_Anomaly_Dataset.tar.gz"
filename = "UCSD_Anomaly_Dataset.tar.gz"

urllib.request.urlretrieve(url, filename)

('UCSD_Anomaly_Dataset.tar.gz', <http.client.HTTPMessage at 0x7f0859b91dc0>)

In [5]:
import tarfile

filename = "UCSD_Anomaly_Dataset.tar.gz"
output_dir = "/content/drive/MyDrive"

with tarfile.open(filename, "r:gz") as tar:
    tar.extractall(output_dir)

In [7]:


from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]


for v in range(1,35):
  imagestore=[]
  video_source_path='/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/'+str(v)
  print(video_source_path)
  fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

  def create_dir(path):
    if not os.path.exists(path):
      os.makedirs(path)

  def remove_old_images(path):
    filelist = glob.glob(os.path.join(path, "*.png"))
    for f in filelist:
      os.remove(f)

  def store(image_path):
	  img=load_img(image_path)
	  img=img_to_array(img)
	#Resize the Image to (227,227,3) for the network to be able to process it. 
	  img=resize(img,(227,227,3))
	#Convert the Image to Grayscale
	  gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]
	  imagestore.append(gray)
#List of all Videos in the Source Directory.
  videos=os.listdir(video_source_path)
  print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
  create_dir(video_source_path+'/frames')

#Remove old images
  remove_old_images(video_source_path+'/frames')

  framepath=video_source_path+'/frames'

  for video in videos:
		  os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		  images=os.listdir(framepath)
		  for image in images:
			  image_path=framepath+ '/'+ image
			  store(image_path)


  imagestore=np.array(imagestore)
  a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
  imagestore.resize(b,c,a)
#Normalize
  imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
  imagestore=np.clip(imagestore,0,1)
  np.save('trainer.npy',imagestore)
#Remove Buffer Directory
  os.system('rm -r {}'.format(framepath))
  print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
  print('Number of frames created :', int(len(imagestore)))


/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/1
Found  202  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/2
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/3
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/4
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227
/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/5
Found  200  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential



def load_model():
	"""
	Return the model used for abnormal event 
	detection in videos using spatiotemporal autoencoder

	"""
	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model





X_train=np.load('trainer.npy')
frames=X_train.shape[2]
frames=frames-frames%10
X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()
epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')


Trainer has been loaded
Epoch 1/200
20/20 [==============================] - ETA: 0s - loss: 0.2408 - accuracy: 0.5700

20/20 [==============================] - 23s 211ms/step - loss: 0.2408 - accuracy: 0.5700
Epoch 2/200
20/20 [==============================] - ETA: 0s - loss: 0.2079 - accuracy: 0.5861

20/20 [==============================] - 4s 219ms/step - loss: 0.2079 - accuracy: 0.5861
Epoch 3/200
20/20 [==============================] - ETA: 0s - loss: 0.1947 - accuracy: 0.6000

20/20 [==============================] - 5s 256ms/step - loss: 0.1947 - accuracy: 0.6000
Epoch 4/200
20/20 [==============================] - ETA: 0s - loss: 0.1870 - accuracy: 0.6065

20/20 [==============================] - 4s 205ms/step - loss: 0.1870 - accuracy: 0.6065
Epoch 5/200
20/20 [==============================] - ETA: 0s - loss: 0.1657 - accuracy: 0.6345

...

20/20 [==============================] - 4s 206ms/step - loss: 0.0212 - accuracy: 0.8429
Epoch 197/200
20/20 [==============================] - ETA: 0s - loss: 0.0212 - accuracy: 0.8429

20/20 [==============================] - 5s 253ms/step - loss: 0.0212 - accuracy: 0.8429
Epoch 198/200
20/20 [==============================] - ETA: 0s - loss: 0.0212 - accuracy: 0.8429

20/20 [==============================] - 5s 227ms/step - loss: 0.0212 - accuracy: 0.8429
Epoch 199/200
20/20 [==============================] - ETA: 0s - loss: 0.0211 - accuracy: 0.8430

20/20 [==============================] - 4s 206ms/step - loss: 0.0211 - accuracy: 0.8430
Epoch 200/200
20/20 [==============================] - ETA: 0s - loss: 0.0211 - accuracy: 0.8430

20/20 [==============================] - 5s 252ms/step - loss: 0.0211 - accuracy: 0.8430
Done
 Please wait while AnomalyDetector.h5 has been created 
Refresh when needed


In [10]:
from tensorflow.keras.utils import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]


for i in range(1,47):
  video_source_path='/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/'+str(i)
  imagestore=[]
  fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

  def create_dir(path):
	  if not os.path.exists(path):
		  os.makedirs(path)

  def remove_old_images(path):
	  filelist = glob.glob(os.path.join(path, "*.png"))
	  for f in filelist:
		  os.remove(f)

  def store(image_path):
	  img=load_img(image_path)
	  img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	  img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	  gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	  imagestore.append(gray)
#List of all Videos in the Source Directory.
  videos=os.listdir(video_source_path)
  print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
  create_dir(video_source_path+'/frames')

#Remove old images
  remove_old_images(video_source_path+'/frames')

  framepath=video_source_path+'/frames'
  total=0
  video_count=0

  for video in videos:
		  video_count+=1
		  print("Video number: ",video_count)
		  print("Video:",str(video))
		  image_count=0
		  os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		  images=os.listdir(framepath)
		  image_count=len(images)
		  for image in images:
			  image_path=framepath+ '/'+ image
			  store(image_path)
		  total=len(images)+total
		  print("Number of images:",image_count,"\n----------\n")


  imagestore=np.array(imagestore)
  a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
  imagestore.resize(b,c,a)
#Normalize
  imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
  imagestore=np.clip(imagestore,0,1)
  np.save('tester.npy',imagestore)
#Remove Buffer Directory
  os.system('rm -r {}'.format(framepath))

  print("Program ended. All testing videos shall be stored in tester.npy \n Please wait while tester.npy is created. \nRefresh when needed")
  print('Number of frames created :', int(total))
  print ('Number of bunches=',int(total),"/10 = ",int(total/10))
  print("\nCorrupted and unreadable bunches were ignored")





Streaming output truncated to the last 5000 lines.

Video number:  11
Video: 141.bmp
Number of images: 1 
----------

Video number:  12
Video: 064.bmp
Number of images: 1 
----------

Video number:  13
Video: 140.bmp
Number of images: 1 
----------

Video number:  14
Video: 023.bmp
Number of images: 1 
----------

Video number:  15
Video: 164.bmp
Number of images: 1 
----------

Video number:  16
Video: 056.bmp
Number of images: 1 
----------

Video number:  17
Video: 112.bmp
Number of images: 1 
----------

Video number:  18
Video: 153.bmp
Number of images: 1 
----------

Video number:  19
Video: 116.bmp
Number of images: 1 
----------

Video number:  20
Video: 157.bmp
Number of images: 1 
----------

Video number:  21
Video: 114.bmp
Number of images: 1 
----------

Video number:  22
Video: 063.bmp
Number of images: 1 
----------

Video number:  23
Video: 145.bmp
Number of images: 1 
----------

Video number:  24
Video: 084.bmp
Number of images: 1 
----------

Video number:  25
Video:

In [11]:
from keras.models import load_model
import numpy as np 




def mean_squared_loss(x1,x2):





	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist








threshold=0.00025 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
#threshold=0.0008#(Accuracy level Vishakha)

model=load_model('AnomalyDetector.h5')

X_test=np.load('tester.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
	
	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)



if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")
	
print("\nCorrupted and unreadable bunches were ignored")

1/1 [==============================] - 1s 706ms/step
No anomaly
bunch number:  1
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  2
1/1 [==============================] - 0s 63ms/step
No anomaly
bunch number:  3
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  4
1/1 [==============================] - 0s 61ms/step
No anomaly
bunch number:  5
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  6
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  7
1/1 [==============================] - 0s 55ms/step
No anomaly
bunch number:  8
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  9
1/1 [==============================] - 0s 60ms/step
No anomaly
bunch number:  10
1/1 [==============================] - 0s 57ms/step
No anomaly
bunch number:  11
1/1 [==============================] - 0s 62ms/step
No anomaly
bunch number:  12
1/1 [==============================]